# Twitter Scraper / API

In [2]:
# Imports here
#import twitter
import json
import tweepy
import geopandas as gpd
import re # regex
import geocoder as gc
import pandas as pd
from shapely.geometry import Point
import os





In [ ]:
# Kuo

import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

#Variables that contains the user credentials to access Twitter API 
access_token = "952108626780422144-DRJswwytxVa2Kqcyn2paSJgITaCCy91"
access_token_secret = "wlGgfyS62lpeH00imJ3dnKc9gzubC3aOAUPvXWr9ac1g3"
consumer_key = "B9199qVuNEMat6QFlyJFiYEFO"
consumer_secret = "n3Uhg5sw5wCcfcKZd5tmjraGdb6AytrudqkHKfgH12nvoY31Ll"

#Variables that contains the user credentials to access Twitter API 
access_token = "952108626780422144-DRJswwytxVa2Kqcyn2paSJgITaCCy91"
access_token_secret = "wlGgfyS62lpeH00imJ3dnKc9gzubC3aOAUPvXWr9ac1g3"
consumer_key = "B9199qVuNEMat6QFlyJFiYEFO"
consumer_secret = "n3Uhg5sw5wCcfcKZd5tmjraGdb6AytrudqkHKfgH12nvoY31Ll"


text_file = open("data/output.txt", "w")
#This is a basic listener that just prints received tweets to stdout.
class StdOutListener(StreamListener):

    def on_data(self, data):
        text_file.write(data)
        return True

    def on_error(self, status):
        print(status)

if __name__ == '__main__':

    #This handles Twitter authetification and the connection to Twitter Streaming API
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    stream = Stream(auth, l)

    # This line filter Twitter Streams to capture data by the keywords and bounding location
    stream.filter(track=['nra'], locations=[-123, 25,-63, 50])





In [3]:
# Kuo
import json
import pandas as pd
import matplotlib.pyplot as plt

tweets_data_path = 'data/output.txt'

tweets_data = []
tweets_file = open(tweets_data_path, "r")
for line in tweets_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue

In [ ]:
# Yulong

consumer_key = 'xcW3f75Y6xEcoKnCrGDrNXSLG'
consumer_secret = 'SNQSDZeDlPoUSa443ElufrWxuCqdljfq6FME7Jin9DZdUQRTKa'
access_token = '153588498-8MoqCHZJG7jrLh8KToTErFAzpOzYMvZkhq1YzYmP'
access_token_secret = 'kw4SF72mY4bv5IuCDQxXQzdkfJMJ7v0t06F2OFMmwMfWT'

"""
api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=consumer_secret,
                  access_token=access_token_key,
                  access_token_secret=access_token_secret)

results = api.GetSearch(
    raw_query='q=%23nra')

print type(results[0])
"""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)


def getTweets(hashtag, count):
    hashtag = '#' + hashtag
    tweets = []
    counter = 0
    for tweet in tweepy.Cursor(api.search, q=hashtag, count=count,
                               lang='en').items():
        if (counter > count):
            print 'Goes over the count'
            break;
        counter += 1
        tweets.append(tweet)
    return tweets

tweets = getTweets('nra', 50)


In [4]:
pattern = re.compile('(^[\w]+,\s\w{2}$)') # City, State

def make_point(lat, lng):
    return Point(lat, lng)

# Creates a folder if it does not exist
def create_folder(name):
    if not os.path.exists(name):
        os.makedirs(name)

def create_gdf(tweets):
    main_list = []
    for t in tweets:
        row = []
        if (t['place']):
            if re.match(pattern, t['place']['full_name']):
                row.append(t['user']['name'])
                row.append(t['user']['screen_name'])
                row.append(t['user']['description'])
                # row.append(t['user']['location'])
                # row.append(t['created_at'])
                row.append(t['text'])
                # row.append(t['retweet_count'])
                # row.append(t['favorite_count'])
                row.append(t['place']['full_name'])
                
                response = gc.osm(t['place']['full_name'] + ', United States')
                row.append(response.y) # lat
                row.append(response.x) # lng
                
                main_list.append(row)
    gdf = gpd.GeoDataFrame(main_list)
    return gdf

gdf = create_gdf(tweets_data)
col_names = ['Name', 'Handle', 'Bio', 'Content', 'Place', 'Lat', 'Lng']
gdf.columns = col_names

gdf['geometry'] = gdf.apply(lambda row: make_point(row['Lat'], row['Lng']), axis = 1)
print gdf



                                                  Name           Handle  \
0                                              Ahhh215      TinoAhhh215   
1                           Alexa ☾ ↠ Spring Gathering         lexrenae   
2                                           sidthekidd      sid10458_xx   
3                                      JUICETHEGOD💲™️    JUICE_STGMB24   
4                                             Mónikiux       Moni_kiux1   
5                                       Anthony Galace    AnthonyGalace   
6                             🤨Siobhán🏳️‍🌈Elizabeth          BhanLiz   
7                                       emily trefsgar        trefffffy   
8                                         Mont Polanco        mpdeuce20   
9                                               RNinTN          RNinTN1   
10                                          ❤BossLady❤       Its_Judyyy   
11                                                 3|3    doitlikedesss   
12                           

In [5]:
# print gdf
gdf.to_csv('data/tweets_with_coordinates.csv', sep = ',', encoding = 'utf-8')

# Save as shapefile
shp_folder = 'shapefiles'
create_folder(shp_folder)
gdf.to_file(shp_folder + '/tweets.shp', driver = 'ESRI Shapefile')